In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from PIL import Image
import os
import torch
from torch import nn
import torch.nn.functional as F
from torch import optim
from torchvision import datasets, models, transforms
import torchvision
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import splitfolders
from datetime import datetime
import yaml
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd


from pipeline_torch_models import *
from pipeline_torch_inception import *

In [3]:
config=yaml.load(open('config.yml', 'r'), Loader=yaml.FullLoader)
seed = config['model_config']['initial_seed']
torch.manual_seed(seed)
np.random.seed(seed)

In [4]:
skip_training = False
LVL_all = False 

### Load Data

In [5]:
if LVL_all == True:
    # define training and test data directories
    data_dir  = r'C:\Users\FINAELB\Documents\Aalto\Dayyan\TL\Data\Scenario 05'
    train_valid_dir = os.path.join(data_dir) 
    #test_dir  = os.path.join(data_dir, 'test')

    splitfolders.ratio(input=train_valid_dir, output='split_data', ratio=(0.6, 0.4))
    train_dir='split_data/train'
    valid_dir='split_data/val'

    splitfolders.ratio(input='split_data/val', output='Valid_Test', ratio=(0.5, 0.5))
    valid_dir='Valid_Test/train'
    test_dir='Valid_Test/val'

    # Selecting mean and std values according to ImageNet dataset
    mean = torch.tensor( [0.485, 0.456, 0.406])
    std = torch.tensor([0.229, 0.224, 0.225])
else: 
    # define training and test data directories# define training and test data directories
    data_dir  = r'C:\Users\FINAELB\Documents\Aalto\Dayyan\TL\Data\Scenario 10\01 Training'
    data_dir_test  = r'C:\Users\FINAELB\Documents\Aalto\Dayyan\TL\Data\Scenario 10\02 Testing'
    train_valid_dir = os.path.join(data_dir) 
    test_dir  = os.path.join(data_dir, 'test')

    splitfolders.ratio(input=train_valid_dir, output='split_data', ratio=(0.6, 0.4))
    train_dir='split_data/train'
    valid_dir='split_data/val'

    test_dir=r'C:\Users\FINAELB\Documents\Aalto\Dayyan\TL\Data\Scenario 10\02 Testing'

    # Selecting mean and std values according to ImageNet dataset
    mean = torch.tensor( [0.485, 0.456, 0.406])
    std = torch.tensor([0.229, 0.224, 0.225])

Copying files: 1888 files [00:22, 82.43 files/s] 


In [6]:
# load and transform data using ImageFolder
data_transforms = {
    'train':  transforms.Compose([
                                transforms.Resize([224,224]),
                                transforms.ToTensor(),
                                transforms.Normalize(mean,std)
                                ]),
    'validation':  transforms.Compose([
                                transforms.Resize([224,224]),
                                transforms.ToTensor(),
                                transforms.Normalize(mean,std)
                                ]),
    'test':  transforms.Compose([
                                transforms.Resize([224,224]),
                                transforms.ToTensor(),
                                transforms.Normalize(mean,std)
                                ])
}

train_data = datasets.ImageFolder(train_dir, transform=data_transforms["train"])
valid_data = datasets.ImageFolder(valid_dir, transform=data_transforms["validation"])
test_data  = datasets.ImageFolder(test_dir, transform=data_transforms["test"])

In [7]:
# load and transform data using ImageFolder
data_transforms_inception = {
    'train_inception':  transforms.Compose([
                                transforms.Resize([299,299]),
                                transforms.ToTensor(),
                                transforms.Normalize(mean,std)
                                ]),
    'validation_inception':  transforms.Compose([
                                transforms.Resize([299,299]),
                                transforms.ToTensor(),
                                transforms.Normalize(mean,std)
                                ]),
    'test_inception':  transforms.Compose([
                                transforms.Resize([299,299]),
                                transforms.ToTensor(),
                                transforms.Normalize(mean,std)
                                ])
}

train_data_inception = datasets.ImageFolder(train_dir, transform=data_transforms_inception["train_inception"])
valid_data_inception = datasets.ImageFolder(valid_dir, transform=data_transforms_inception["validation_inception"])
test_data_inception  = datasets.ImageFolder(test_dir, transform=data_transforms_inception["test_inception"])

### Load models and prepare for TL

In [8]:
## MODEL 1: Resnet 18
model_1 = models.resnet18(pretrained=True)
layers=list(model_1._modules.keys())

layers_frozen=layers[0:8]

for layer in layers_frozen:
    for param in model_1._modules[layer].parameters():
        param.requires_grad=False
        
# modify last layer to match it our classes
n_inputs = model_1.fc.in_features
last_layer = nn.Linear(n_inputs, len(train_data.classes))
model_1.fc = last_layer

model_1 = models.resnet18(pretrained=True)
layers=list(model_1._modules.keys())

layers_frozen=layers[0:8]

for layer in layers_frozen:
    for param in model_1._modules[layer].parameters():
        param.requires_grad=False
        
# modify last layer to match it our classes
n_inputs = model_1.fc.in_features
last_layer = nn.Linear(n_inputs, len(train_data.classes))
model_1.fc = last_layer

model_1 = model_1.to('cuda')

c:\Users\FINAELB\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\FINAELB\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [9]:
## MODEL 2: Resnet 50
model_2 = models.resnet50(pretrained=True)
layers=list(model_2._modules.keys())

layers_frozen=layers[0:8]

for layer in layers_frozen:
    for param in model_2._modules[layer].parameters():
        param.requires_grad=False
        
# modify last layer to match it our classes
n_inputs = model_2.fc.in_features
last_layer = nn.Linear(n_inputs, len(train_data.classes))
model_2.fc = last_layer

model_2 = model_2.to('cuda')

c:\Users\FINAELB\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [10]:
## MODEL 3: VGG 16

model_3 = models.vgg16(pretrained=True)
layers=list(model_3._modules.keys())

layers_frozen=layers[0:30]

for layer in layers_frozen:
    for param in model_3._modules[layer].parameters():
        param.requires_grad=False

# modify last layer to match it our classes
n_inputs = model_3.classifier[6].in_features
last_layer = nn.Linear(n_inputs, len(train_data.classes))
model_3.classifier[6] = last_layer

model_3 = model_3.to('cuda')

c:\Users\FINAELB\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [11]:
# MODEL 4: AlexNet

model_4 = models.alexnet(pretrained=True)
layers=list(model_4._modules.keys())

layers_frozen=layers[0:12]

for layer in layers_frozen:
    for param in model_4._modules[layer].parameters():
        param.requires_grad=False

# modify last layer to match it our classes
n_inputs = model_4.classifier[6].in_features
last_layer = nn.Linear(n_inputs, len(train_data.classes))
model_4.classifier[6] = last_layer

model_4 = model_4.to('cuda')

c:\Users\FINAELB\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [12]:
# MODEL 5: GoogleNet

model_5 = models.googlenet(pretrained=True)
layers=list(model_5._modules.keys())

layers_frozen=layers[0:16]

for layer in layers_frozen:
    for param in model_5._modules[layer].parameters():
        param.requires_grad=False

# modify last layer to match it our classes

n_inputs = model_5.fc.in_features
last_layer = nn.Linear(n_inputs, len(train_data.classes))
model_5.fc = last_layer

model_5 = model_5.to('cuda')

c:\Users\FINAELB\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [13]:
# MODEL 6: Inception
model_6 = models.inception_v3(pretrained=True)
layers=list(model_6._modules.keys())

layers_frozen=layers[0:19]

for layer in layers_frozen:
    for param in model_6._modules[layer].parameters():
        param.requires_grad

# modify last layer to match it our classes
n_inputs = model_6.fc.in_features
last_layer = nn.Linear(n_inputs, len(train_data.classes))
model_6.fc = last_layer

model_6 = model_6.to('cuda')

c:\Users\FINAELB\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


### Training

In [14]:
def rename_model_file(old_name, new_name, directory='models'):
    # Construct full file paths
    old_file_path = os.path.join(directory, old_name)
    new_file_path = os.path.join(directory, new_name)
    
    # Check if the old file exists
    if os.path.exists(old_file_path):
        # Rename the file
        os.rename(old_file_path, new_file_path)
        print(f"File renamed from {old_name} to {new_name}")
    else:
        print(f"File {old_name} does not exist in the directory {directory}")

In [15]:
pipeline1 = PipelineTorch(model_1, config)
if skip_training == False:
    pipeline1.train(train_data, valid_data, config['model_config']['version'])
    # Example usage
    old_file_name = 'model_' + config['model_config']['version'] + '.pth'
    new_file_name = 'model1_' + config['model_config']['version'] + '.pth'
    rename_model_file(old_file_name, new_file_name) 

Epoch [1/25], Learning Rate: 0.000907
Duration of training at epoch 1 is : 0:00:21.654950 seconds.
Epoch: 1 	Training Loss: 1.357278 	Validation Loss: 1.230704
Validation loss has descreased (inf-->1.230704). Saving model...
Epoch [2/25], Learning Rate: 0.000797
Duration of training at epoch 2 is : 0:00:11.522637 seconds.
Epoch: 2 	Training Loss: 1.166948 	Validation Loss: 1.059178
Validation loss has descreased (1.230704-->1.059178). Saving model...
Epoch [3/25], Learning Rate: 0.000659
Duration of training at epoch 3 is : 0:00:11.487635 seconds.
Epoch: 3 	Training Loss: 1.049008 	Validation Loss: 0.999853
Validation loss has descreased (1.059178-->0.999853). Saving model...
Epoch [4/25], Learning Rate: 0.000504
Duration of training at epoch 4 is : 0:00:11.272810 seconds.
Epoch: 4 	Training Loss: 0.964990 	Validation Loss: 0.971981
Validation loss has descreased (0.999853-->0.971981). Saving model...
Epoch [5/25], Learning Rate: 0.000350
Duration of training at epoch 5 is : 0:00:13.38

In [16]:
pipeline2 = PipelineTorch(model_2, config)
if skip_training == False: 
    pipeline2.train(train_data, valid_data, config['model_config']['version'])
    # Example usage
    old_file_name = 'model_' + config['model_config']['version'] + '.pth'
    new_file_name = 'model2_' + config['model_config']['version'] + '.pth'
    rename_model_file(old_file_name, new_file_name)

Epoch [1/25], Learning Rate: 0.000907
Duration of training at epoch 1 is : 0:00:11.602132 seconds.
Epoch: 1 	Training Loss: 1.323315 	Validation Loss: 1.222650
Validation loss has descreased (inf-->1.222650). Saving model...
Epoch [2/25], Learning Rate: 0.000797
Duration of training at epoch 2 is : 0:00:11.656367 seconds.
Epoch: 2 	Training Loss: 1.112048 	Validation Loss: 1.100108
Validation loss has descreased (1.222650-->1.100108). Saving model...
Epoch [3/25], Learning Rate: 0.000659
Duration of training at epoch 3 is : 0:00:11.792472 seconds.
Epoch: 3 	Training Loss: 0.982494 	Validation Loss: 0.905227
Validation loss has descreased (1.100108-->0.905227). Saving model...
Epoch [4/25], Learning Rate: 0.000504
Duration of training at epoch 4 is : 0:00:12.307852 seconds.
Epoch: 4 	Training Loss: 0.910128 	Validation Loss: 0.855856
Validation loss has descreased (0.905227-->0.855856). Saving model...
Epoch [5/25], Learning Rate: 0.000350
Duration of training at epoch 5 is : 0:00:12.24

In [17]:
pipeline3 = PipelineTorch(model_3, config)
if skip_training == False: 
    pipeline3.train(train_data, valid_data, config['model_config']['version'])
    old_file_name = 'model_' + config['model_config']['version'] + '.pth'
    new_file_name = 'model3_' + config['model_config']['version'] + '.pth'
    rename_model_file(old_file_name, new_file_name)

Epoch [1/25], Learning Rate: 0.000907
Duration of training at epoch 1 is : 0:00:14.992119 seconds.
Epoch: 1 	Training Loss: 1.429179 	Validation Loss: 1.266027
Validation loss has descreased (inf-->1.266027). Saving model...
Epoch [2/25], Learning Rate: 0.000797
Duration of training at epoch 2 is : 0:00:32.699790 seconds.
Epoch: 2 	Training Loss: 1.336876 	Validation Loss: 1.186913
Validation loss has descreased (1.266027-->1.186913). Saving model...
Epoch [3/25], Learning Rate: 0.000659
Duration of training at epoch 3 is : 0:00:27.148982 seconds.
Epoch: 3 	Training Loss: 1.220159 	Validation Loss: 1.180158
Validation loss has descreased (1.186913-->1.180158). Saving model...
Epoch [4/25], Learning Rate: 0.000504
Duration of training at epoch 4 is : 0:00:27.457396 seconds.
Epoch: 4 	Training Loss: 1.251424 	Validation Loss: 1.164181
Validation loss has descreased (1.180158-->1.164181). Saving model...
Epoch [5/25], Learning Rate: 0.000350
Duration of training at epoch 5 is : 0:00:27.92

In [18]:
pipeline4 = PipelineTorch(model_4, config)
if skip_training == False: 
    pipeline4.train(train_data, valid_data, config['model_config']['version'])
    old_file_name = 'model_' + config['model_config']['version'] + '.pth'
    new_file_name = 'model4_' + config['model_config']['version'] + '.pth'
    rename_model_file(old_file_name, new_file_name) 

Epoch [1/25], Learning Rate: 0.000907
Duration of training at epoch 1 is : 0:00:15.165866 seconds.
Epoch: 1 	Training Loss: 1.517926 	Validation Loss: 1.414316
Validation loss has descreased (inf-->1.414316). Saving model...
Epoch [2/25], Learning Rate: 0.000797
Duration of training at epoch 2 is : 0:00:14.632850 seconds.
Epoch: 2 	Training Loss: 1.449749 	Validation Loss: 1.457431
seed has been changed. The new torch seed is 20
Epoch [3/25], Learning Rate: 0.000659
Duration of training at epoch 3 is : 0:00:15.201380 seconds.
Epoch: 3 	Training Loss: 1.429688 	Validation Loss: 1.334543
Validation loss has descreased (1.414316-->1.334543). Saving model...
Epoch [4/25], Learning Rate: 0.000504
Duration of training at epoch 4 is : 0:00:13.900713 seconds.
Epoch: 4 	Training Loss: 1.462822 	Validation Loss: 1.390813
seed has been changed. The new torch seed is 40
Epoch [5/25], Learning Rate: 0.000350
Duration of training at epoch 5 is : 0:00:15.456530 seconds.
Epoch: 5 	Training Loss: 1.414

In [19]:
pipeline5 = PipelineTorch(model_5, config)
if skip_training == False: 
    pipeline5.train(train_data, valid_data, config['model_config']['version'])
    old_file_name = 'model_' + config['model_config']['version'] + '.pth'
    new_file_name = 'model5_' + config['model_config']['version'] + '.pth'
    rename_model_file(old_file_name, new_file_name)     

Epoch [1/25], Learning Rate: 0.000907
Duration of training at epoch 1 is : 0:00:16.112891 seconds.
Epoch: 1 	Training Loss: 1.362739 	Validation Loss: 1.289792
Validation loss has descreased (inf-->1.289792). Saving model...
Epoch [2/25], Learning Rate: 0.000797
Duration of training at epoch 2 is : 0:00:15.580293 seconds.
Epoch: 2 	Training Loss: 1.215248 	Validation Loss: 1.157542
Validation loss has descreased (1.289792-->1.157542). Saving model...
Epoch [3/25], Learning Rate: 0.000659
Duration of training at epoch 3 is : 0:00:16.214869 seconds.
Epoch: 3 	Training Loss: 1.143358 	Validation Loss: 1.114215
Validation loss has descreased (1.157542-->1.114215). Saving model...
Epoch [4/25], Learning Rate: 0.000504
Duration of training at epoch 4 is : 0:00:15.916175 seconds.
Epoch: 4 	Training Loss: 1.090414 	Validation Loss: 1.052225
Validation loss has descreased (1.114215-->1.052225). Saving model...
Epoch [5/25], Learning Rate: 0.000350
Duration of training at epoch 5 is : 0:00:15.85

In [20]:
pipeline6 = PipelineTorchInception(model_6, config)
if skip_training == False:
    pipeline6.train(train_data_inception, valid_data_inception, config['model_config']['version'])
    old_file_name = 'model_' + config['model_config']['version'] + '.pth'
    new_file_name = 'model6_' + config['model_config']['version'] + '.pth'
    rename_model_file(old_file_name, new_file_name)

Epoch [1/25], Learning Rate: 0.000907
Duration of training at epoch 1 is : 0:02:52.788070 seconds.
Epoch: 1 	Training Loss: 0.666084 	Validation Loss: 2.804018
Validation loss has descreased (inf-->2.804018). Saving model...
Epoch [2/25], Learning Rate: 0.000797
Duration of training at epoch 2 is : 0:02:32.468964 seconds.
Epoch: 2 	Training Loss: 0.379814 	Validation Loss: 0.610710
Validation loss has descreased (2.804018-->0.610710). Saving model...
Epoch [3/25], Learning Rate: 0.000659
Duration of training at epoch 3 is : 0:02:20.821934 seconds.
Epoch: 3 	Training Loss: 0.195391 	Validation Loss: 2.710434
seed has been changed. The new torch seed is 30
Epoch [4/25], Learning Rate: 0.000504
Duration of training at epoch 4 is : 0:02:19.389639 seconds.
Epoch: 4 	Training Loss: 0.056632 	Validation Loss: 0.336693
Validation loss has descreased (0.610710-->0.336693). Saving model...
Epoch [5/25], Learning Rate: 0.000350
Duration of training at epoch 5 is : 0:02:30.807771 seconds.
Epoch: 5

### Post_Processing

In [21]:
pipeline1.load_checkpoint(config['model_config']['version'],nb_model=1)
pipeline2.load_checkpoint(config['model_config']['version'],nb_model=2)
pipeline3.load_checkpoint(config['model_config']['version'],nb_model=3)
pipeline4.load_checkpoint(config['model_config']['version'],nb_model=4)
pipeline5.load_checkpoint(config['model_config']['version'],nb_model=5)
pipeline6.load_checkpoint(config['model_config']['version'],nb_model=6)

=> loading checkpoint 'models\model1_v11.pth'
=> loaded checkpoint 'models\model1_v11.pth' (epoch 25)
=> loading checkpoint 'models\model2_v11.pth'


c:\Users\FINAELB\Documents\Aalto\Dayyan\TL\TL_CWT\DL_classification\pipeline_torch_models.py:151: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint=torch.load(os.path

=> loaded checkpoint 'models\model2_v11.pth' (epoch 25)
=> loading checkpoint 'models\model3_v11.pth'
=> loaded checkpoint 'models\model3_v11.pth' (epoch 15)
=> loading checkpoint 'models\model4_v11.pth'
=> loaded checkpoint 'models\model4_v11.pth' (epoch 7)
=> loading checkpoint 'models\model5_v11.pth'
=> loaded checkpoint 'models\model5_v11.pth' (epoch 25)
=> loading checkpoint 'models\model6_v11.pth'


c:\Users\FINAELB\Documents\Aalto\Dayyan\TL\TL_CWT\DL_classification\pipeline_torch_inception.py:149: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint=torch.load(os.p

=> loaded checkpoint 'models\model6_v11.pth' (epoch 9)


In [22]:
losses1_test, average_loss1_test, predictions1_test, real_labels1_test, acc1_test= pipeline1.predict(test_data)   
losses2_test, average_loss2_test, predictions2_test, real_labels2_test, acc2_test= pipeline2.predict(test_data)   
losses3_test, average_loss3_test, predictions3_test, real_labels3_test, acc3_test= pipeline3.predict(test_data)   
losses4_test, average_loss4_test, predictions4_test, real_labels4_test, acc4_test= pipeline4.predict(test_data)   
losses5_test, average_loss5_test, predictions5_test, real_labels5_test, acc5_test= pipeline5.predict(test_data)
losses6_test, average_loss6_test, predictions6_test, real_labels6_test, acc6_test= pipeline6.predict(test_data_inception)   


In [23]:
losses1_valid, average_loss1_valid, predictions1_valid, real_labels1_valid, acc1_valid= pipeline1.predict(valid_data)   
losses2_valid, average_loss2_valid, predictions2_valid, real_labels2_valid, acc2_valid= pipeline2.predict(valid_data)   
losses3_valid, average_loss3_valid, predictions3_valid, real_labels3_valid, acc3_valid= pipeline3.predict(valid_data)   
losses4_valid, average_loss4_valid, predictions4_valid, real_labels4_valid, acc4_valid= pipeline4.predict(valid_data)   
losses5_valid, average_loss5_valid, predictions5_valid, real_labels5_valid, acc5_valid= pipeline5.predict(valid_data)  
losses6_valid, average_loss6_valid, predictions6_valid, real_labels6_valid, acc6_valid= pipeline6.predict(valid_data_inception)

In [24]:

losses1_train, average_loss1_train, predictions1_train, real_labels1_train, acc1_train= pipeline1.predict(train_data)   
losses2_train, average_loss2_train, predictions2_train, real_labels2_train, acc2_train= pipeline2.predict(train_data)   
losses3_train, average_loss3_train, predictions3_train, real_labels3_train, acc3_train= pipeline3.predict(train_data)   
losses4_train, average_loss4_train, predictions4_train, real_labels4_train, acc4_train= pipeline4.predict(train_data)   
losses5_train, average_loss5_train, predictions5_train, real_labels5_train, acc5_train= pipeline5.predict(train_data) 
losses6_train, average_loss6_train, predictions6_train, real_labels6_train, acc6_train= pipeline6.predict(train_data_inception) 

In [25]:
# Data from the pipeline predictions (replace these with your actual outputs)
train_accuracies = [acc1_train[0], acc2_train[0], acc3_train[0], acc4_train[0], acc5_train[0], acc6_train[0]]
valid_accuracies = [acc1_valid[0], acc2_valid[0], acc3_valid[0], acc4_valid[0], acc5_valid[0], acc6_valid[0]]
test_accuracies = [acc1_test[0], acc2_test[0], acc3_test[0], acc4_test[0], acc5_test[0], acc6_test[0]]

train_losses = [average_loss1_train, average_loss2_train, average_loss3_train, average_loss4_train, average_loss5_train, average_loss6_train]
valid_losses = [average_loss1_valid, average_loss2_valid, average_loss3_valid, average_loss4_valid, average_loss5_valid, average_loss6_valid]
test_losses = [average_loss1_test, average_loss2_test, average_loss3_test, average_loss4_test, average_loss5_test, average_loss6_test]

# Creating a DataFrame with the organized data
df = pd.DataFrame({
    'Model': ['Model 1', 'Model 2', 'Model 3', 'Model 4', 'Model 5', 'Model 6'],
    'Train Accuracy': train_accuracies,
    'Validation Accuracy': valid_accuracies,
    'Test Accuracy': test_accuracies,
    'Train Loss': train_losses,
    'Validation Loss': valid_losses,
    'Test Loss': test_losses
})

In [26]:
df

,Model,Train Accuracy,Validation Accuracy,Test Accuracy,Train Loss,Validation Loss,Test Loss
0,Model 1,85.381356,85.830619,88.559322,0.522318,0.529665,0.464089
1,Model 2,84.576271,82.980456,85.381356,0.503816,0.551837,0.489256
2,Model 3,65.042373,67.345277,71.822034,0.993737,0.981733,0.948867
3,Model 4,50.593220,49.185668,43.432203,1.313484,1.312813,1.327955
4,Model 5,77.457627,75.407166,74.364407,0.684670,0.733525,0.730239
5,Model 6,99.915254,99.837134,99.576271,0.000957,0.004141,0.021215
